In [2]:
import os
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')  # Needed for POS tagging
import re
import gensim
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer, WordNetLemmatizer
from gensim.corpora import Dictionary
from gensim.models import LdaModel, Phrases, TfidfModel, CoherenceModel
from nltk.corpus import wordnet
import pyLDAvis.gensim_models
from nltk import pos_tag

if __name__ == '__main__':   
    data_dir = '/Users/juanpablocasado/Documents/Digital text/Digital Text (f)/cleaned3'  # your directory

    documents = []
    for dirpath, dirnames, filenames in os.walk(data_dir):
        print(f"Looking in directory: {dirpath}")
        for file in filenames:
            print(f"Loading file: {file}")
            with open(os.path.join(dirpath, file), 'r', errors='ignore') as f:
                text = f.read()
                documents.append(text)

    #print(f"Number of documents loaded: {len(documents)}")

    # Debug: Print the first document
    #print(f"First document: {documents[0]}")

    def clean_document(doc):
        # Remove punctuation
        text = re.sub('[^a-zA-Z]', ' ', doc)
        # Convert to lowercase
        text = text.lower()
        # Split into words
        words = text.split()
        # Remove stopwords
        words = [w for w in words if not w in set(stopwords.words('english'))]
        return words

    documents = [clean_document(doc) for doc in documents]

    # Debug: Print the first cleaned document
    #print(f"First cleaned document: {documents[0]}")

    stemmer = PorterStemmer()
    lemmatizer = WordNetLemmatizer()

    def lemmatize(words):
        lemmatizer = WordNetLemmatizer()
        tagged_words = pos_tag(words)
        lemmatized = [lemmatizer.lemmatize(word) for word, pos in tagged_words if pos.startswith('N') or pos.startswith('J')]
        return lemmatized

    documents = [lemmatize(doc) for doc in documents]

    # Debug: Print the first stemmed and lemmatized document
    #print(f"First stemmed and lemmatized document: {documents[0]}")

    bigram = Phrases(documents, min_count=5)
    trigram = Phrases(bigram[documents], min_count=3)

    documents = [trigram[bigram[doc]] for doc in documents]

    # Debug: Print the first document with bigrams
    #print(f"First document with bigrams: {documents[0]}")

    # Create a dictionary representation of the documents
    dictionary = gensim.corpora.Dictionary(documents)

    #print(f"Initial dictionary: {dictionary}")

    # Filter out words that occur less than 10 documents, or more than 70% of the documents
    dictionary.filter_extremes(no_below=10, no_above=0.8)
    print(f"Dictionary after filtering extremes: {dictionary}")

    # Bag-of-words representation of the documents
    corpus = [dictionary.doc2bow(doc) for doc in documents]

    # Debug: Print the first BoW representation
    print(f"First BoW representation: {corpus[0]}")

    # Invert the word-to-id mapping to get id2word
    id2word = {v: k for k, v in dictionary.token2id.items()}

    # Set training parameters
    num_topics = 10  # number of topics, this can be tuned later
    chunksize = 2000
    passes = 20
    iterations = 400
    eval_every = 1
    max_coherence = -1
    best_model = None
    best_num_topics = None

    # Train the LDA model
    for num_topics in range(2, 21):  # try num_topics from 2 to 20
        model = LdaModel(
            corpus=corpus,
            id2word=id2word,
            chunksize=chunksize,
            alpha='auto',
            eta='auto',
            iterations=iterations,
            num_topics=num_topics,
            passes=passes,
            eval_every=eval_every
        )

        # Compute c_v coherence score
        coherence_model_cv = CoherenceModel(model=model, texts=documents, dictionary=dictionary, coherence='c_v')
        coherence_cv = coherence_model_cv.get_coherence()

        if coherence_cv > max_coherence:
            max_coherence = coherence_cv
            best_model = model
            best_num_topics = num_topics

    print(f'Number of topics: {num_topics}, Coherence Score: {coherence_cv}')

    print(f'Best Model: {best_num_topics} topics, Coherence Score: {max_coherence}')

    # Print the topics learned by the model
    print("Topics learned by the LDA model:")
    topics = model.print_topics()
    for topic in topics:
        print(topic)

    with open('/Users/juanpablocasado/Documents/Digital text/Digital Text (f)/topics.txt', 'w') as f:
        for topic in topics:
            f.write(str(topic) + '\n')

    with open('/Users/juanpablocasado/Documents/Digital text/Digital Text (f)/topics_per_document.txt', 'w') as f:
        for idx, doc in enumerate(documents):
            doc_bow = dictionary.doc2bow(doc)
            doc_topics = model.get_document_topics(doc_bow)
            f.write(f"Document {idx}: {doc_topics}\n")

    # Compute u_mass coherence score
    coherence_model_umass = CoherenceModel(model=model, texts=documents, dictionary=dictionary, coherence='u_mass')
    coherence_umass = coherence_model_umass.get_coherence()
    print('Coherence Score (u_mass): ', coherence_umass)

    # Compute c_v coherence score
    coherence_model_cv = CoherenceModel(model=model, texts=documents, dictionary=dictionary, coherence='c_v')
    coherence_cv = coherence_model_cv.get_coherence()
    print('Coherence Score (c_v): ', coherence_cv)

    print("Script completed successfully.")
    



[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/juanpablocasado/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/juanpablocasado/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /Users/juanpablocasado/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


Looking in directory: /Users/juanpablocasado/Documents/Digital text/Digital Text (f)/cleaned3
Loading file: Property in Digital Cultural Heritage Assets.txt
Loading file: Rationales for the Legal Protection of Intangible Goods and Cultural Heritage.txt
Loading file: .DS_Store
Loading file: Council of Europe Convention on Offences relating to Cultural Property.txt
Loading file: European Cultural Convention.docx.txt
Loading file: Notes on Criteria Cultural Heritage.txt
Loading file: 070920-en-AHEG First Draft Recommendation-final.txt
Loading file: Council of Europe Framework Conv .txt
Loading file: The Evolution of Cultural Heritage Ethics.txt
Loading file: The future of (international) cultural heritage law.txt
Loading file: New Policies and Rules for Digital Preservation.txt
Loading file: Moral Rights in the Digital Age New Possibilities for the Democratization of Culture.txt
Loading file: Priorities for the Work Plan for Culture 2023-2026.txt
Loading file: Criteria Cultural Heritage.t

In [3]:
    # Visualize the topics
    pyLDAvis.enable_notebook()
    vis = pyLDAvis.gensim_models.prepare(best_model, corpus, dictionary)
    vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
3     -0.060239  0.082785       1        1  31.134021
1      0.004740  0.013175       2        1  18.997066
4      0.000857  0.029043       3        1  17.482888
0      0.108191 -0.109630       4        1  13.503054
5      0.078462  0.069990       5        1  11.256228
2     -0.132012 -0.085364       6        1   7.626744, topic_info=                  Term        Freq       Total Category  logprob  loglift
47   cultural_heritage  443.000000  443.000000  Default  30.0000  30.0000
152           property  685.000000  685.000000  Default  29.0000  29.0000
43          convention  294.000000  294.000000  Default  28.0000  28.0000
172              right  283.000000  283.000000  Default  27.0000  27.0000
117       member_state  146.000000  146.000000  Default  26.0000  26.0000
..                 ...         ...         ...      ...      ...      ...
22                   c   13.170326  183.928617   Topic6  -4.6848  -0.0631
114            measure   10.633544   88.871002   Topic6  -4.8988   0.4503
103                law   11.812487  230.292077   Topic6  -4.7936  -0.3967
136         particular   10.327250   95.528825   Topic6  -4.9280   0.3489
146          principle   10.129779  127.574064   Topic6  -4.9473   0.0403

[304 rows x 6 columns], token_table=      Topic      Freq    Term
term                         
0         1  0.024734  access
0         2  0.478181  access
0         3  0.107179  access
0         4  0.041223  access
0         5  0.329780  access
...     ...       ...     ...
205       1  0.592591    year
205       2  0.272009    year
205       3  0.068002    year
205       4  0.029144    year
205       6  0.029144    year

[1033 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[4, 2, 5, 1, 6, 3])